In [ ]:
!pip install -q cassio datasets langchain openai tiktoken

In [ ]:
from langchain.vectorstores.cassandra import Cassandra 
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings

In [ ]:
from datasets import load_dataset
import cassio

In [ ]:
!pip install PyPDF2

In [ ]:
from PyPDF2 import PdfReader
import os

In [ ]:
ASTRA_DB_APPLICATION_TOKEN = "AstraCS:tPYgbhoqidssSmlSgcmIFdSq:"
ASTRA_DB_ID = ""

os.environ["OPENAI_API_TYPE"] = "azure"
 
os.environ["OPENAI_API_VERSION"] = "2023-07-01-preview"
 
os.environ["AZURE_OPENAI_API_KEY"] = ""
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://openai-nonfunsbx-pprd-01.openai.azure.com/openai/deployments/gpt-4-finance/chat/completions?api-version=2023-07-01-preview"

In [ ]:
pdfreader = PdfReader(r"C:\Users\SESA737860\Desktop\llms\INN_IND_2024WESOP_EN_30 04 2024.pdf")

In [ ]:
from typing_extensions import Concatenate

raw_text = ''
for i, page in enumerate(pdfreader.pages):
    content = page.extract_text()
    if content:
        raw_text += page.extract_text()

In [ ]:
raw_text

In [ ]:

with open('C:\\Users\\SESA737860\\Downloads\\technical_spec_ocr(2).txt', 'r') as f:
    raw_text = f.read()
    print(raw_text)
    

In [ ]:
cassio.init(token=ASTRA_DB_APPLICATION_TOKEN, database_id=ASTRA_DB_ID)

In [ ]:
from langchain.chat_models.azure_openai import AzureChatOpenAI
llm = AzureChatOpenAI(
    openai_api_version="2023-07-01-preview",
    azure_deployment="gpt-4",
    model_version="gpt-4"
)

In [ ]:
llm

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2"
)

In [ ]:
astra_vector_store = Cassandra(
    embedding = embeddings,
    table_name = "chatbotdemo",
    session = None,
    keyspace = None,
)

In [ ]:
with open(r'C:\Users\SESA737860\Desktop\llms\using_markdown_testimage.txt', 'r') as file:
    filetext = file.read()

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 800,
    chunk_overlap = 200,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

In [ ]:
texts[:50]

In [ ]:
len(texts)

In [ ]:
astra_vector_store.add_texts(texts[50:122])
# print("inserted %i headlines into the vector store" % len(texts[:50]))
print("inserted %i headlines into the vector store" % len(texts[:50]))

In [ ]:
astra_vector_index = VectorStoreIndexWrapper(vectorstore=astra_vector_store)

In [ ]:
first_question = True
while True:
    if first_question:
        query_text = input("\n enter your quesion (or type quit to exit): ").strip()
    else:
        query_text = input("\n enter your follow-up question (or type quit to exit): ").strip()

    if query_text.lower() == "quit":
        break
    
    if query_text == "":
        print("Please enter a question")
        continue

    first_question = False

    print("\n Question: \"%s\"" % query_text)
    answer = astra_vector_index.query(query_text, llm).strip()
    print("ANSWER: \"%s\"\n" % answer)

    print("documents by relevance:")
    for doc, score in astra_vector_store.similarity_search_with_score(query_text, k=4):
        print("[%0.4f] \"%s ...\"" % (score, doc.page_content))
